<a href="https://colab.research.google.com/github/AxelRK200/trend_finder_dataiku/blob/main/Exploration/Donchian_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install ta

In [31]:
import pandas as pd
import yfinance as yf
from ta.volatility import DonchianChannel

# Import et mise en forme #

In [32]:
ndx = yf.Ticker("^NDX")
df_DC = ndx.history(period="max")
# Passe la date d'index à colonne
df_DC.reset_index(inplace=True)

In [33]:
df_DC.drop(columns=['Volume','Dividends','Stock Splits'], inplace=True)

In [34]:
cols_to_round = df_DC.columns.to_list()
print(cols_to_round)

['Date', 'Open', 'High', 'Low', 'Close']


In [35]:
for i in range (1,5):
  df_DC[cols_to_round[i]] = df_DC[cols_to_round[i]].apply(lambda f : round(f,2))

In [36]:
df_DC["Date"] = df_DC.Date.apply(lambda d : d.strftime('%Y-%m-%d'))

# Indicateur technique #

In [37]:
df_DC["DC_up"] = DonchianChannel(high=df_DC.High, low=df_DC.Low, close=df_DC.Close, window=10).donchian_channel_hband()
df_DC["DC_low"] = DonchianChannel(high=df_DC.High, low=df_DC.Low, close=df_DC.Close, window=10).donchian_channel_lband()
df_DC["DC_width"] = DonchianChannel(high=df_DC.High, low=df_DC.Low, close=df_DC.Close, window=10).donchian_channel_wband()

In [38]:
df_DC.drop(columns=['High','Open','Low'], inplace=True)
df_DC.dropna(inplace=True)
df_DC.reset_index(drop=True, inplace=True)

In [40]:
df_DC.head()

,Date,Close,DC_up,DC_low,DC_width
0,1985-10-14,113.18,113.27,106.75,5.918932
1,1985-10-15,113.38,113.98,106.75,6.556099
2,1985-10-16,114.32,114.34,106.75,6.860769
3,1985-10-17,114.74,115.34,106.75,7.737623
4,1985-10-18,114.32,115.34,106.75,7.708115


# Feature Engineering #

In [41]:
df_DC.DC_width = df_DC.DC_width.apply(lambda f : round(f,2))

In [47]:
df_DC["DC_up_lag"] = df_DC.DC_up.shift(1)
df_DC["DC_low_lag"] = df_DC.DC_low.shift(1)
df_DC["DC_width_lag2_diff"] = df_DC.DC_width - df_DC.DC_width.shift(2)
df_DC["DC_width_lag5_diff"] = df_DC.DC_width - df_DC.DC_width.shift(5)

In [57]:
def breakout_signal(df):
  ''' Finds if Close breaks upper or lower donchian channel '''

  df_calc = df.copy()

  s_Close_DCupLag_diff = df_calc.Close - df_calc.DC_up_lag
  s_Close_DClowLag_diff = df_calc.Close - df_calc.DC_low_lag

  s_Close_DCupLag_diff = [1 if e>0 else 0 for e in s_Close_DCupLag_diff]
  s_Close_DClowLag_diff = [-1 if e<0 else 0 for e in s_Close_DClowLag_diff]

  return s_Close_DCupLag_diff, s_Close_DClowLag_diff

In [58]:
df_DC["breakout_upper_channel"], df_DC["breakout_lower_channel"] = breakout_signal(df_DC)

In [59]:
df_DC.tail()

,Date,Close,DC_up,DC_low,DC_width,DC_up_lag,DC_low_lag,DC_width_lag2_diff,DC_width_lag5_diff,breakout_upper_channel,breakout_lower_channel
9391,2023-01-18,11410.29,11690.27,10696.42,8.84,11615.95,10696.42,1.18,3.04,0,0
9392,2023-01-19,11295.67,11690.27,10696.42,8.81,11690.27,10696.42,0.59,2.14,0,0
9393,2023-01-20,11619.03,11690.27,10696.42,8.75,11690.27,10696.42,-0.09,1.48,0,0
9394,2023-01-23,11872.54,11918.86,11050.68,7.58,11690.27,10696.42,-1.23,-0.08,1,0
9395,2023-01-24,11864.80,11918.86,11050.68,7.53,11918.86,11050.68,-1.22,-0.69,0,0


In [60]:
df_DC.describe()

,Close,DC_up,DC_low,DC_width,DC_up_lag,DC_low_lag,DC_width_lag2_diff,DC_width_lag5_diff,breakout_upper_channel,breakout_lower_channel
count,9396.000000,9396.000000,9396.000000,9396.000000,9395.000000,9395.000000,9394.000000,9391.000000,9396.000000,9396.000000
mean,2868.546588,2949.744027,2760.217472,6.811008,2948.789358,2759.335039,0.000280,0.000716,0.161771,-0.075245
std,3478.806164,3576.416869,3350.895596,4.237497,3575.409746,3349.981976,1.662692,2.781374,0.368260,0.263800
min,113.180000,113.270000,106.750000,1.160000,113.270000,106.750000,-22.040000,-26.630000,0.000000,-1.000000
25%,415.932500,419.980000,403.440000,4.080000,419.980000,403.440000,-0.620000,-1.350000,0.000000,0.000000
50%,1628.075000,1690.045000,1572.440000,5.640000,1689.680000,1572.440000,0.000000,0.010000,0.000000,0.000000
75%,3645.037500,3780.822500,3489.367500,8.290000,3780.430000,3486.160000,0.610000,1.280000,0.000000,0.000000
max,16573.340000,16764.860000,16012.780000,46.880000,16764.860000,16012.780000,19.540000,26.970000,1.000000,0.000000


In [62]:
df_DC.loc[df_DC["breakout_lower_channel"] == -1].shape[0]

707

In [63]:
df_DC.loc[df_DC["breakout_upper_channel"] == 1].shape[0]

1520